<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 1

Run date time (IST): 2023-07-07 15:31:15


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP', 'CAMS',
       'CDSL', 'CLEAN', 'DMART', 'EMAMILTD', 'FINEORG', 'GICRE',
       'GILLETTE', 'GLAND', 'HDFCAMC', 'ICICIGI', 'INDIGOPNTS',
       'INDUSINDBK', 'INFY', 'IRCTC', 'ITBEES', 'JMFINANCIL', 'JUBLFOOD',
       'LTIM', 'LUXIND', 'METROPOLIS', 'NIACL', 'PAGEIND', 'PIDILITIND',
       'PNB', 'RAJESHEXPO', 'RELAXO', 'RELIANCE', 'ROSSARI', 'STARHEALTH',
       'SUNTV', 'TCS', 'TEAMLEASE', 'UJJIVANSFB', 'VAIBHAVGBL',
       'VALIANTORG', 'WIPRO'], dtype=object)

In [6]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [7]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 46
Investment:  2896317.0
Current:  3031669.0
Today PnL:  -14439.0
Today PnL %:  -0.47
Current PnL:  135352.0
Current PnL %:  4.67
Estimate:  5256166.0
Estimate PnL:  2359849.0
Estimate PnL %:  81.48


In [9]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
42,41.95,UJJIVANSFB.NS,57.0,41.10,34.87,1350.0,V40N,ATH,47074.50,56632.50,55485.00,76950.0,2.07,20.30,63.46,43.16,1.63,33.0
18,2250.05,HDFCAMC.NS,3518.0,2259.75,1879.66,41.0,V40,ATH,77066.06,92252.05,92649.75,144238.0,-0.43,19.71,87.16,67.45,2.66,52.0
33,60.50,PNB.NS,95.0,59.00,50.85,900.0,SS,ATH,45765.00,54450.00,53100.00,85500.0,2.54,18.98,86.82,67.84,1.58,30.0
19,1332.85,ICICIGI.NS,1613.0,1340.20,1137.80,69.0,V40,ATH,78508.20,91966.65,92473.80,111297.0,-0.55,17.14,41.76,24.62,2.71,46.0
0,399.00,5PAISA.NS,553.0,402.50,347.06,228.0,V40N,ATH,79129.68,90972.00,91770.00,126084.0,-0.87,14.97,59.34,44.37,2.73,41.0
10,1176.00,CDSL.NS,1648.0,1190.90,1027.05,49.0,SS,ATH,50325.45,57624.00,58354.10,80752.0,-1.25,14.50,60.46,45.96,1.74,25.0
16,5070.40,GILLETTE.NS,7529.0,5130.95,4475.85,21.0,V40,ATH,93992.85,106478.40,107749.95,158109.0,-1.18,13.28,68.21,54.93,3.25,43.0
1,1562.30,AAVAS.NS,3196.0,1537.65,1390.60,23.0,2T3Y,2T3Y,31983.80,35932.90,35365.95,73508.0,1.60,12.35,129.83,117.48,1.10,14.0
2,699.00,AMARAJABAT.NS,988.0,703.35,625.25,72.0,V200,ATH,45018.00,50328.00,50641.20,71136.0,-0.62,11.80,58.02,46.22,1.55,18.0
8,4645.90,BAYERCROP.NS,6002.0,4651.30,4185.24,11.0,V40N,ATH,46037.64,51104.90,51164.30,66022.0,-0.12,11.01,43.41,32.40,1.59,18.0


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
4,3346.75,ASIANPAINT.NS,3542.0,3399.40,3091.34,25.0,V40,ATH,77283.50,83668.75,84985.00,88550.0,-1.55,8.26,14.58,6.32,2.67,22.0
36,2629.00,RELIANCE.NS,2811.0,2638.75,2467.15,31.0,V40,ATH,76481.65,81499.00,81801.25,87141.0,-0.37,6.56,13.94,7.38,2.64,17.0
32,2619.85,PIDILITIND.NS,2902.0,2649.00,2504.06,30.0,V40,ATH,75121.80,78595.50,79470.00,87060.0,-1.10,4.62,15.89,11.27,2.59,12.0
40,3321.00,TCS.NS,3857.0,3322.90,3231.00,16.0,V40,ATH,51696.00,53136.00,53166.40,61712.0,-0.06,2.79,19.37,16.58,1.78,5.0
5,1612.05,BAJAJFINSV.NS,1905.0,1619.90,1454.93,52.0,V40,ATH,75656.36,83826.60,84234.80,99060.0,-0.48,10.80,30.93,20.13,2.61,28.0
19,1332.85,ICICIGI.NS,1613.0,1340.20,1137.80,69.0,V40,ATH,78508.20,91966.65,92473.80,111297.0,-0.55,17.14,41.76,24.62,2.71,46.0
24,31.08,ITBEES.NS,40.0,30.94,30.07,1675.0,SS,ATH,50367.25,52059.00,51824.50,67000.0,0.45,3.36,33.02,29.66,1.74,6.0
8,4645.90,BAYERCROP.NS,6002.0,4651.30,4185.24,11.0,V40N,ATH,46037.64,51104.90,51164.30,66022.0,-0.12,11.01,43.41,32.40,1.59,18.0
7,1669.00,BATAINDIA.NS,2173.0,1673.50,1523.62,100.0,V40,ATH,152362.00,166900.00,167350.00,217300.0,-0.27,9.54,42.62,33.08,5.26,50.0
3,417.95,AMBUJACEM.NS,581.0,429.00,424.02,180.0,TBD,ATH,76323.60,75231.00,77220.00,104580.0,-2.58,-1.43,37.02,38.45,2.64,-4.0


In [11]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
17,1047.00,GLAND.NS,4316.0,1060.45,1225.87,55.0,2T3Y,2T3Y,67422.85,57585.00,58324.75,237380.0,-1.27,-14.59,252.08,266.67,2.33,-34.0
44,539.90,VALIANTORG.NS,1851.0,544.70,542.52,144.0,2T3Y,2T3Y,78122.88,77745.60,78436.80,266544.0,-0.88,-0.48,241.19,241.67,2.70,-1.0
28,1500.00,LUXIND.NS,4546.0,1500.70,1493.72,51.0,V40N,2T3Y,76179.72,76500.00,76535.70,231846.0,-0.05,0.42,204.34,203.92,2.63,1.0
43,310.00,VAIBHAVGBL.NS,966.0,308.45,317.48,144.0,2T3Y,2T3Y,45717.12,44640.00,44416.80,139104.0,0.50,-2.36,204.27,206.63,1.58,-4.0
6,229.80,BANDHANBNK.NS,723.0,230.70,253.98,300.0,2T3Y,2T3Y,76194.00,68940.00,69210.00,216900.0,-0.39,-9.52,184.67,194.19,2.63,-25.0
30,118.50,NIACL.NS,303.0,118.25,117.07,390.0,TBD,ATH,45657.30,46215.00,46117.50,118170.0,0.21,1.22,158.82,157.60,1.58,2.0
29,1454.00,METROPOLIS.NS,3440.0,1473.70,1365.98,22.0,TBD,ATH,30051.56,31988.00,32421.40,75680.0,-1.34,6.44,151.83,145.39,1.04,7.0
41,2384.00,TEAMLEASE.NS,5281.0,2423.90,2279.93,21.0,V40N,2T3Y,47878.53,50064.00,50901.90,110901.0,-1.65,4.56,131.63,127.07,1.65,8.0
1,1562.30,AAVAS.NS,3196.0,1537.65,1390.60,23.0,2T3Y,2T3Y,31983.80,35932.90,35365.95,73508.0,1.60,12.35,129.83,117.48,1.10,14.0
20,1445.00,INDIGOPNTS.NS,3112.0,1449.25,1407.73,124.0,V40N,2T3Y,174558.52,179180.00,179707.00,385888.0,-0.29,2.65,121.07,118.42,6.03,16.0


In [12]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
20,1445.00,INDIGOPNTS.NS,3112.0,1449.25,1407.73,124.0,V40N,2T3Y,174558.52,179180.00,179707.00,385888.0,-0.29,2.65,121.07,118.42,6.03,16.0
7,1669.00,BATAINDIA.NS,2173.0,1673.50,1523.62,100.0,V40,ATH,152362.00,166900.00,167350.00,217300.0,-0.27,9.54,42.62,33.08,5.26,50.0
31,36602.00,PAGEIND.NS,53549.0,36988.80,39325.03,3.0,V40,ATH,117975.09,109806.00,110966.40,160647.0,-1.05,-6.92,36.17,43.09,4.07,-28.0
16,5070.40,GILLETTE.NS,7529.0,5130.95,4475.85,21.0,V40,ATH,93992.85,106478.40,107749.95,158109.0,-1.18,13.28,68.21,54.93,3.25,43.0
0,399.00,5PAISA.NS,553.0,402.50,347.06,228.0,V40N,ATH,79129.68,90972.00,91770.00,126084.0,-0.87,14.97,59.34,44.37,2.73,41.0
19,1332.85,ICICIGI.NS,1613.0,1340.20,1137.80,69.0,V40,ATH,78508.20,91966.65,92473.80,111297.0,-0.55,17.14,41.76,24.62,2.71,46.0
44,539.90,VALIANTORG.NS,1851.0,544.70,542.52,144.0,2T3Y,2T3Y,78122.88,77745.60,78436.80,266544.0,-0.88,-0.48,241.19,241.67,2.70,-1.0
4,3346.75,ASIANPAINT.NS,3542.0,3399.40,3091.34,25.0,V40,ATH,77283.50,83668.75,84985.00,88550.0,-1.55,8.26,14.58,6.32,2.67,22.0
26,494.00,JUBLFOOD.NS,903.0,492.45,471.56,164.0,SS,ATH,77335.84,81016.00,80761.80,148092.0,0.31,4.76,91.49,86.73,2.67,13.0
12,3820.00,DMART.NS,5324.0,3815.90,3671.87,21.0,SS,ATH,77109.27,80220.00,80133.90,111804.0,0.11,4.03,44.99,40.96,2.66,11.0


In [13]:
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

Mounted at /content/drive


In [14]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
17,1047.00,GLAND.NS,4316.0,1060.45,1225.87,55.0,2T3Y,2T3Y,67422.85,57585.00,58324.75,237380.0,-1.27,-14.59,252.08,266.67,2.33,-34.0
31,36602.00,PAGEIND.NS,53549.0,36988.80,39325.03,3.0,V40,ATH,117975.09,109806.00,110966.40,160647.0,-1.05,-6.92,36.17,43.09,4.07,-28.0
6,229.80,BANDHANBNK.NS,723.0,230.70,253.98,300.0,2T3Y,2T3Y,76194.00,68940.00,69210.00,216900.0,-0.39,-9.52,184.67,194.19,2.63,-25.0
34,526.00,RAJESHEXPO.NS,949.0,529.30,624.78,48.0,V40N,ATH,29989.44,25248.00,25406.40,45552.0,-0.62,-15.81,51.89,67.70,1.04,-16.0
43,310.00,VAIBHAVGBL.NS,966.0,308.45,317.48,144.0,2T3Y,2T3Y,45717.12,44640.00,44416.80,139104.0,0.50,-2.36,204.27,206.63,1.58,-4.0
3,417.95,AMBUJACEM.NS,581.0,429.00,424.02,180.0,TBD,ATH,76323.60,75231.00,77220.00,104580.0,-2.58,-1.43,37.02,38.45,2.64,-4.0
11,1361.00,CLEAN.NS,2648.0,1374.30,1396.49,23.0,2T3Y,2T3Y,32119.27,31303.00,31608.90,60904.0,-0.97,-2.54,89.62,92.16,1.11,-3.0
23,629.30,IRCTC.NS,1164.0,632.30,633.05,120.0,SS,ATH,75966.00,75516.00,75876.00,139680.0,-0.47,-0.59,83.87,84.46,2.62,-2.0
9,2283.00,CAMS.NS,3867.0,2301.30,2324.85,13.0,SS,ATH,30223.05,29679.00,29916.90,50271.0,-0.80,-1.80,66.33,68.13,1.04,-2.0
44,539.90,VALIANTORG.NS,1851.0,544.70,542.52,144.0,2T3Y,2T3Y,78122.88,77745.60,78436.80,266544.0,-0.88,-0.48,241.19,241.67,2.70,-1.0


In [15]:
print('Total profit: ',round(sum(tmp_df[tmp_df['Current P/L%'] > 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] > 0]['Investment']),0))
print('Total loss: ',round(sum(tmp_df[tmp_df['Current P/L%'] < 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] < 0]['Investment']),0))

Total profit:  169826.0
Total loss:  -34474.0


In [16]:
# SIP based on portfolio
tmp_df[tmp_df['Weightage%'] < 2].sort_values(by = 'Current P/L%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
34,526.00,RAJESHEXPO.NS,949.0,529.30,624.78,48.0,V40N,ATH,29989.44,25248.0,25406.40,45552.0,-0.62,-15.81,51.89,67.70,1.04,-16.0
11,1361.00,CLEAN.NS,2648.0,1374.30,1396.49,23.0,2T3Y,2T3Y,32119.27,31303.0,31608.90,60904.0,-0.97,-2.54,89.62,92.16,1.11,-3.0
43,310.00,VAIBHAVGBL.NS,966.0,308.45,317.48,144.0,2T3Y,2T3Y,45717.12,44640.0,44416.80,139104.0,0.50,-2.36,204.27,206.63,1.58,-4.0
9,2283.00,CAMS.NS,3867.0,2301.30,2324.85,13.0,SS,ATH,30223.05,29679.0,29916.90,50271.0,-0.80,-1.80,66.33,68.13,1.04,-2.0
13,421.00,EMAMILTD.NS,588.0,426.95,422.59,72.0,V200,ATH,30426.48,30312.0,30740.40,42336.0,-1.39,-0.38,39.14,39.52,1.05,-0.0
38,600.95,STARHEALTH.NS,907.0,597.05,599.50,50.0,ATH,NaN,29975.00,30047.5,29852.50,45350.0,0.65,0.24,51.29,51.05,1.03,0.0
30,118.50,NIACL.NS,303.0,118.25,117.07,390.0,TBD,ATH,45657.30,46215.0,46117.50,118170.0,0.21,1.22,158.82,157.60,1.58,2.0
14,4809.00,FINEORG.NS,7196.0,4826.30,4697.43,11.0,V40N,ATH,51671.73,52899.0,53089.30,79156.0,-0.36,2.38,53.19,50.81,1.78,4.0
27,5115.70,LTIM.NS,7439.0,5178.20,4996.37,10.0,V200,ATH,49963.70,51157.0,51782.00,74390.0,-1.21,2.39,48.89,46.50,1.73,4.0
37,865.30,ROSSARI.NS,1569.0,857.15,844.40,36.0,2T3Y,2T3Y,30398.40,31150.8,30857.40,56484.0,0.95,2.48,85.81,83.33,1.05,3.0
